In [10]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [16]:
from qutip import Qobj, identity, sigmax, sigmay, sigmaz, tensor,num
from qutip.operators import num,position,momentum
from qutip.qip.gates import swap
from qutip.qip.algorithms import qft
import qutip.logging_utils as logging
logger = logging.get_logger()
#Set this to None or logging.WARN for 'quiet' execution
log_level = logging.INFO
#QuTiP control modules
import qutip.control.pulseoptim as cpo
import qutip.control.pulsegen as pulsegen

example_name = 'SWAP'

In [17]:
Nho = 10
omega1 =0.01
omega2 =0.01
omega =2*omega1
Nt=10
tunit = 0.01
tmax = tunit*Nt

In [18]:
### Defining the physics


In [29]:
Q = identity(2)
F = identity(Nho)


# Drift Hamiltonian
H_d = omega1*tensor(sigmaz(),Q,F) +\
     omega2*tensor(Q,sigmaz(),F) +\
     omega*tensor(Q,Q,num(Nho))
# Control Hamiltonian     
H_c= [tensor(sigmax(),Q,position(Nho)) +\
     tensor(Q,sigmax(),position(Nho))]
n_ctrls = len(H_c) #1    
    
# start point for the gate evolution
U_0 = tensor(identity(2),identity(2),identity(Nho))


# Target for gate the evolution 
U_targ = tensor(swap(),identity(Nho))     


Defining the time evolution parameters

In [30]:
# Number of time slots
n_ts = 200
# Time allowed for the evolution
evo_time = 10

Set the conditions which will cause the pulse optimisation to terminate

In [31]:
# Fidelity error target
fid_err_targ = 1e-3
# Maximum iterations for the optisation algorithm
max_iter = 20000
# Maximum (elapsed) time allowed in seconds
max_wall_time = 300

Give an extension for output files


In [32]:
#Set to None to suppress output files
f_ext = "{}_n_ts{}.txt".format(example_name, n_ts)

Create the optimiser objects

In [36]:
optim = cpo.create_pulse_optimizer(H_d, H_c, U_0, U_targ, n_ts, evo_time, 
                fid_err_targ=fid_err_targ, 
                max_iter=max_iter, max_wall_time=max_wall_time,
                alg='CRAB', 
                dyn_type='UNIT', 
                prop_type='DIAG', 
                fid_type='UNIT', fid_params={'phase_option':'PSU'}, 
                log_level=log_level, gen_stats=True)
                

Configure the pulses for each of the controls

In [ ]:
dyn = optim.dynamics

# Control 
crab_pgen = optim.pulse_generator[0]
# Start from a sine wave
guess_pgen = pulsegen.create_pulse_gen('SIN', dyn=dyn, 
                                           pulse_params={'scaling':1.0})
crab_pgen.guess_pulse = guess_pgen.gen_pulse()
crab_pgen.scaling = 0.0
# Add some higher frequency components
crab_pgen.num_coeffs = 5

# Control 1
crab_pgen = optim.pulse_generator[0]
# Start from a ramped pulse
guess_pgen = pulsegen.create_pulse_gen('LIN', dyn=dyn, 
                                           pulse_params={'scaling':3.0})
crab_pgen.guess_pulse = guess_pgen.gen_pulse()
crab_pgen.scaling = 0.0
# Add some higher frequency components
crab_pgen.num_coeffs = 5

# Control 2
crab_pgen = optim.pulse_generator[1]
# Apply a ramping pulse that will force the start and end to zero
ramp_pgen = pulsegen.create_pulse_gen('GAUSSIAN_EDGE', dyn=dyn, 
                                    pulse_params={'decay_time':evo_time/50.0})
crab_pgen.ramping_pulse = ramp_pgen.gen_pulse()

# Control 3
crab_pgen = optim.pulse_generator[2]
# Add bounds
crab_pgen.scaling = 0.5
crab_pgen.lbound = -2.0
crab_pgen.ubound = 2.0


# Control 4
crab_pgen = optim.pulse_generator[3]
# Start from a triangular pulse with small signal
guess_pgen = pulsegen.PulseGenTriangle(dyn=dyn)
guess_pgen.num_waves = 1
guess_pgen.scaling = 2.0
guess_pgen.offset = 2.0
crab_pgen.guess_pulse = guess_pgen.gen_pulse()
crab_pgen.scaling = 0.1

init_amps = np.zeros([n_ts, n_ctrls])
for j in range(dyn.num_ctrls):
    pgen = optim.pulse_generator[j]
    pgen.init_pulse()
    init_amps[:, j] = pgen.gen_pulse()

dyn.initialize_controls(init_amps)